## This notebook is for calling functions to train models and run prediction

In [ ]:
import sys
import os
os.chdir("..")

import numpy as np
from orientation_detection.NN_module import get_CNN, calculate_accuracy, calculate_angle_accuracy, get_FCNN, train_model, get_prediction,plot_learning_curves
from orientation_detection.render_images import get_images, get_train_val, show_images
from keras.applications.vgg16 import preprocess_input
import keras
import time
import datetime
import cv2
import pickle
from keras.utils import plot_model

In [ ]:
root_logdir = os.path.join(os.curdir, "my_logs")
logdir = os.path.join(root_logdir, "run_{}".format(time.time()))

## Set general parameters and get Training and test data

In [ ]:
## set parameters
#batch_size defines the number of samples for one forward and back propagation
#epochs  is number of one forward pass and backward pass of all the training batches
target_type = 'vectorOrientations'
image_size = 64
num_images = 10000

# Get Training and Validation data (Can choose from saved training images or render new images)
rendered_images = get_images(num_images,image_size)
#rendered_images = pickle.load(open('train_shine_10000_blur07.pkl','rb'))

# convert into x_train, xVal, yTrain and yVal
xTrain, xVal, yTrain, yVal= get_train_val(rendered_images,target_type)

#Getting the images for testing (Can choose from saved test images or render new images)
image_test = get_images(1000,image_size)
#with open('test_shine_'+str(len(image_test['images']))+'_blur07.pkl','wb') as f:   
#    pickle.dump(image_test, f)

x_test = np.array(image_test['images'])
y_test = np.array([np.concatenate(x) for x in image_test['vectorOrientations']])

# 1. CNN
### 1.1 Build model and save

### Parameters set up

In [ ]:
parameters_conv = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'batch_size':30,
    'epochs':50,
    'conv':[[96,11,'relu'],[384,5,'relu'],[256,5,'relu']],
    'padding':['same','same'],  ## padding for conv layer 1 Try with padding same or valid
    'maxpool':[2,2,2],  
    'FC':[[256,'relu'],[512,'relu']],
    'dropout':[0.0,0.2],
    'output_activation':'linear',
    'optimizer': 'rmsprop' 
}

parameters_conv2 = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'batch_size':30,
    'epochs':50,
    'conv':[[96,11,'relu'],[384,5,'relu'],[256,5,'relu']],
    'padding':['same','same'],  ## padding for conv layer 1 and 2 Try with padding same or valid
    'maxpool':[2,2,2],  
    'FC':[[1024,'relu'],[256,'relu']],
    'dropout':[0.5,0.3],
    'output_activation':'linear',
    'optimizer': 'rmsprop' 
}

# can change the list to whichever parameter we want to train the model with
parameters_CNN = [parameters_conv,parameters_conv2]

### Run the training model for every parameter set

In [ ]:
for parameter in parameters_CNN:
    #filename for the name your model and history
    filename = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
    ML_type = 'conv'

    #run the training
    history = train_model(filename,np.array(xTrain), np.array(xVal), yTrain, yVal, ML_type,parameter)

    #plot the learning_curves
    plot_learning_curves(history)

### 1.2 CNN Run Prediction with selected model and show pair of images (test image and prediction)
- will also return RMSE and angle discrepancy

In [ ]:
#Load saved model
#Please change the model_filename to load the correct model
model_filename = '../trained_model/new_chair/CNN/CNN_model_2019-04-13 22:002019-04-13 22:00'

#Run Prediction
predictions = get_prediction(target_type,model_filename,x_test, y_test)

#print images from pair of true data and prediction, will also return the RMSE and Angle Discrepancy
show_images(y_test,predictions,x_test,target_type,image_size)


# Fully Connected Neural Network
### 2.1 Build model and save

In [ ]:
param_FC = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'batch_size':64,
    'epochs':50,
    'FC':[[4096,'relu'],[1024,'relu'],[256,'relu']],
    'dropout':[0.0,0.0,0.5],
    'output_activation':'linear',
    'optimizer': 'adam'
}
parameters_FC = [param_FC] 

### Run the training model for every parameter set

In [ ]:
for parameter in parameters_FC:
    #filename for the name your model and history
    filename = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

    xTrain_fc = np.array([x.flatten() for x in xTrain])
    xVal_fc = np.array([x.flatten() for x in xVal])
    ML_type = 'fc'

    #run the training 
    history_fc = train_model(filename,xTrain_fc, xVal_fc, yTrain, yVal, ML_type,parameter)

    #plot the learning_curves
    plot_learning_curves(history_fc)

### 2.2 FC Run Prediction with selected model and show pair of images (test image and prediction)
- will also return RMSE and angle discrepancy

In [ ]:
#Load saved model
#Please change the model_filename to load the correct model
model_filename_fc = "trained_model/FC_model_2019-04-12 10:492019-04-12 10:49"

x_test_fc = np.array([x.flatten() for x in image_test['images']])

#Run Prediction
predictions_fc = get_prediction(target_type,model_filename_fc,x_test_fc, y_test)

#print images from pair of true data and prediction
show_images(y_test,predictions_fc,x_test_fc,target_type,image_size)

#Just validation of manually calculated mse and RMSE
mse,rmse = calculate_accuracy(y_test,predictions_fc)
print("Just for crosschecking, mse and rmse calculated outside of the keras")
print('mse = ', mse)
print('rmse =', rmse)

# 3. VGG16
### 3.1 Build model and save

In [ ]:

parameters_vgg1 = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'not_freeze': 4, #specify number of vgg which is not going to be freezed for training, 0 means freeze all
    'pre_trained':True,
    'batch_size':64,
    'epochs':30,
    'FC':[[1024,'relu'],[512,'relu']],
    'dropout':[0.0,0.3],
    'output_activation':'linear',
    'optimizer': 'adam', 
    'learning_rate': 'default',
    'preprocessing':'yes',
    'loss':'custom',
    'note':''
}

parameters_vgg2 = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'not_freeze': 0, #specify number of vgg which is not going to be freezed for training, 0 means freeze all
    'pre_trained':True,
    'batch_size':64,
    'epochs':30,
    'FC':[[1024,'relu'],[512,'relu']],
    'dropout':[0.0,0.3],
    'output_activation':'tanh',
    'optimizer': 'adam', 
    'learning_rate': 'default',
    'preprocessing':'yes',
    'loss':'mean_squared_error',
    'note':'best parameter, using mean squared error and tanh'
}


parameters_group= [parameters_vgg1, parameters_vgg2]

## To preprocess the image before using vgg16 network
xTrain_copy = np.array(xTrain).copy()
# prepare the image for the VGG model
xTrain_copy = preprocess_input(xTrain_copy)

xVal_copy = np.array(xVal).copy()
# prepare the image for the VGG model
xVal_copy = preprocess_input(xVal_copy)


In [ ]:
## With preprocessing
for parameter in parameters_group:
    #filename for the name your model and history 
    filename = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
    ML_type = 'vgg16'

    #run the training
    history = train_model(filename,xTrain_copy, xVal_copy, yTrain, yVal, ML_type,parameter)
        
    #plot the learning_curves
    plot_learning_curves(history)

### VGG Run Prediction with selected model and show pair of images 
- will also return RMSE and angle discrepancy

In [ ]:
#Load saved model
#Please change the model_filename to load the correct model
model_filename='../trained_model/new_chair/vgg16/vgg16_model_2019-04-18 20:07'

#load test data from saved pickle file
image_test = pickle.load(open('../Octiba-Nima_files/training data/test_200.pkl','rb'))
x_test = np.array(image_test['images'])
y_test = np.array([np.concatenate(x) for x in image_test['vectorOrientations']])


x_test_copy = x_test.copy()
# prepare the image for the VGG model
x_test_copy = preprocess_input(x_test_copy)

#Run Prediction
predictions = get_prediction(target_type,model_filename,x_test_copy, y_test)

#print images from pair of true data and prediction
show_images(y_test,predictions,x_test,target_type,image_size)

# 4. Resnet50
### 4.1 Build model and save

In [ ]:

parameters_resnet50_1 = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'not_freeze': 21 , #specify number of vgg which is not going to be freezed for training
    'pre_trained':True,
    'batch_size':64,
    'epochs':20,
    'output_activation':'linear',
    'optimizer': 'sgd', ## Changed to adamax from adam
    'learning_rate': 0.0001,
    'preprocessing':'yes'
}

parameters_resnet50_2 = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'not_freeze': 12, #specify number of vgg which is not going to be freezed for training
    'pre_trained':True,
    'batch_size':64,
    'epochs':20,
    'output_activation':'linear',
    'optimizer': 'sgd', ## Changed to adamax from adam
    'learning_rate': 0.0001,
    'preprocessing':'yes'
}

parameters_resnet50_3 = {
    'image_size':image_size,
    'num_images': num_images,
    'target_type':'vectorOrientations',
    'not_freeze': 16, #specify number of vgg which is not going to be freezed for training
    'pre_trained':True,
    'batch_size':64,
    'epochs':50,
    'output_activation':'linear',
    'optimizer': 'sgd', ## Changed to adamax from adam
    'learning_rate': 0.001,
    'preprocessing':'yes'
}
parameters_res = [parameters_resnet50_1,parameters_resnet50_2]


## To preprocess the image before using ResNet-50 network
xTrain_copy2 = np.array(xTrain).copy()
# Preprocess input using preprocess_input from keras.applications.restnet50
xTrain_copy2 = keras.applications.resnet50.preprocess_input(xTrain_copy2)

xVal_copy2 = np.array(xVal).copy()
# prepare the image for the VGG model
xVal_copy2 = keras.applications.resnet50.preprocess_input(xVal_copy2)

In [ ]:
# Run the training for each set of parameter
for parameter in parameters_res:
    
    filename = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
    ML_type = 'resnet50'

    #run the training and prediction
    history = train_model(filename,xTrain_copy2, xVal_copy2, yTrain, yVal, ML_type,parameter)
    
    #plot the learning_curves
    plot_learning_curves(history)

## To test the model on real Test Data

In [ ]:
import matplotlib.pyplot as plt

## Getting the x_test
arr = os.listdir('../cropped_image/')
#cropped = [cv2.resize(np.load('../cropped_image/'+x)[0],(32,32)) for x in arr]
cropped = [np.load('../cropped_image/'+x)[0] for x in arr]
x_test = [cv2.cvtColor(x,cv2.COLOR_BGR2RGB) for x in cropped]

#Load saved model
#Please change the model_filename to load the correct model
model_filename = '../trained_model/new_chair/vgg16/To_be_tested_16_april/2019-04-15 23:53/vgg16_model_2019-04-15 23:53'

#Documents/Myfolders/Lectures/DataMining/trained_model/new_chair/vgg16/To_be_tested_16_april/2019-04-15 23:53/vgg16_model_2019-04-15 23:53
model = load_CNN_model('vectorOrientations',model_filename)
#Run Prediction
predictions = model.predict(preprocess_input(np.array(x_test).copy()))

for i in range(len(x_test)):
    print('====================================================================================================')
    plt.imshow(x_test[i].reshape(image_size,image_size,3))
    plt.show()
    predicted_image = get_images(1,image_size,vectorOrientation=predictions[i].reshape(2,3))
    plt.imshow(np.array(predicted_image['images']).astype(int).reshape(image_size,image_size,3))    
    plt.show()
    print('====================================================================================================')